<a href="https://colab.research.google.com/github/vitthal-bhandari/Detecting-Homophobia-and-Transphobia-2.0/blob/master/baselines/English_mBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize environment

In [1]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar  6 09:30:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install datasets

     |████████████████████████████████| 311 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 80.7 MB/s 
     |████████████████████████████████| 134 kB 71.5 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 1.1 MB 84.3 MB/s 
     |████████████████████████████████| 271 kB 85.7 MB/s 
     |████████████████████████████████| 94 kB 4.9 MB/s 
     |████████████████████████████████| 144 kB 74.4 MB/s 


In [4]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 6.5 MB 59.1 MB/s 
     |████████████████████████████████| 895 kB 73.1 MB/s 
     |████████████████████████████████| 596 kB 55.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [174]:
!pip3 install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html
     |██████████████████████████████▎ | 834.1 MB 85.5 MB/s eta 0:00:01tcmalloc: large alloc 1147494400 bytes == 0x5633919ea000 @  0x7fe78cc37615 0x5633582d53bc 0x5633583b618a 0x5633582d81cd 0x5633583cab3d 0x56335834c458 0x56335834702f 0x5633582d9aba 0x56335834c2c0 0x56335834702f 0x5633582d9aba 0x563358348cd4 0x5633583cb986 0x563358348350 0x5633583cb986 0x563358348350 0x5633583cb986 0x563358348350 0x5633582d9f19 0x56335831da79 0x5633582d8b32 0x56335834c1dd 0x56335834702f 0x5633582d9aba 0x563358348cd4 0x56335834702f 0x5633582d9aba 0x563358347eae 0x5633582d99da 0x563358348108 0x56335834702f
     |████████████████████████████████| 881.9 MB 2.0 kB/s 
     |████████████████████████████████| 23.2 MB 1.2 MB/s 
     |████████████████████████████████| 2.9 MB 40.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch

In [191]:
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np
import torch

# Load data

In [151]:
eng_train_dataset = load_dataset("csv", data_files="eng_3_train.tsv", sep="\t")
eng_train = eng_train_dataset["train"]
eng_dev_dataset = load_dataset("csv", data_files="eng_3_dev.tsv", sep="\t")
eng_dev = eng_dev_dataset["train"]
eng_test_dataset = load_dataset("csv", data_files="eng_3_test.tsv", sep="\t")
eng_test = eng_test_dataset["train"]
print(len(eng_train), len(eng_dev), len(eng_test))

Using custom data configuration default-b001170a93d0c76e
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-b001170a93d0c76e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-64ac90cb2c154475
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-64ac90cb2c154475/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-4d716ae01a2ba992
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-4d716ae01a2ba992/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

3164 792 999


In [152]:
tam_train_dataset = load_dataset("csv", data_files="tam_3_train.tsv", sep="\t")
tam_train = tam_train_dataset["train"]
tam_dev_dataset = load_dataset("csv", data_files="tam_3_dev.tsv", sep="\t")
tam_dev = tam_dev_dataset["train"]
tam_test_dataset = load_dataset("csv", data_files="tam_3_test.tsv", sep="\t")
tam_test = tam_test_dataset["train"]
print(len(tam_train), len(tam_dev), len(tam_test))

Using custom data configuration default-658f0ad647ebed4a
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-658f0ad647ebed4a/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-c7a461aa2c56ec03
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-c7a461aa2c56ec03/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-3215af4694f3e61e
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-3215af4694f3e61e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

2662 666 833


In [153]:
eng_tam_train_dataset = load_dataset("csv", data_files="eng_tam_3_train.tsv", sep="\t")
eng_tam_train = eng_tam_train_dataset["train"]
eng_tam_dev_dataset = load_dataset("csv", data_files="eng_tam_3_dev.tsv", sep="\t")
eng_tam_dev = eng_tam_dev_dataset["train"]
eng_tam_test_dataset = load_dataset("csv", data_files="eng_tam_3_test.tsv", sep="\t")
eng_tam_test = eng_tam_test_dataset["train"]
print(len(eng_tam_train), len(eng_tam_dev), len(eng_tam_test))

Using custom data configuration default-b1ffd50211c4056c
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-b1ffd50211c4056c/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-af4b1b8788eb5ed1
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-af4b1b8788eb5ed1/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-66509fdbbcb927fd
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-66509fdbbcb927fd/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

3861 966 1207


In [154]:

eng_train_dataset["train"] = eng_train_dataset["train"].remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])
eng_train_dataset["train"] = eng_train_dataset["train"].rename_column(original_column_name= 'category', new_column_name= 'label')
eng_train_dataset["train"] = eng_train_dataset["train"].class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-b001170a93d0c76e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-6c8f7769527a1b29.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-b001170a93d0c76e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-a719a2bc55efea51.arrow


In [155]:
eng_train = eng_train.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'])
eng_train = eng_train.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_train = eng_train.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-b001170a93d0c76e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-6c8f7769527a1b29.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-b001170a93d0c76e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-a719a2bc55efea51.arrow


In [156]:
eng_dev = eng_dev.remove_columns(column_names=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
                                       'Unnamed: 9'])
eng_dev = eng_dev.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_dev = eng_dev.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-64ac90cb2c154475/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-ce726c50cba8ffe2.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-64ac90cb2c154475/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-1ab46d17d30d820d.arrow


In [157]:
eng_test = eng_test.rename_column(original_column_name= 'category', new_column_name= 'label')
eng_test = eng_test.rename_column(original_column_name= 'text                        ', new_column_name= 'text')
eng_test = eng_test.class_encode_column(column="label")

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-4d716ae01a2ba992/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-8e0b9031e6419d9a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-4d716ae01a2ba992/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-5585ec35a24fa4f2.arrow


In [158]:
eng_test

Dataset({
    features: ['label', 'text'],
    num_rows: 999
})

In [159]:
# remove NA rows in test set
eng_test = eng_test.filter(lambda example, indice: indice < 990, with_indices=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-4d716ae01a2ba992/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-a74e8f7f09b0a284.arrow


In [160]:
eng_test

Dataset({
    features: ['label', 'text'],
    num_rows: 990
})

**Converting Datasets to Datafrane**

In [161]:
eng_train.set_format(type="pandas")
eng_dev.set_format(type="pandas")
eng_test.set_format(type="pandas")

In [162]:
df_eng_train = eng_train[:]
df_eng_dev = eng_dev[:]
df_eng_test = eng_test[:]

df_eng_test

,label,text
0,1,Spr....2016 poitan feel happy with my partner ...
1,1,R u still with ur partner
2,1,excellent movie..no unnecessary drama or scene...
3,1,"For those who speak about culture., pre marita..."
4,1,Best movie and people not understand relations...
...,...,...
985,1,Looks like Karthik took advantage on Varun whe...
986,1,i am really crying pro😢😢😢😭😭😭😭😭😭 ...
987,1,They may be transgender but don't ever forgot ...
988,1,It is their own choice. I support them No doub...


In [163]:
def label_int2str(row):
  return eng_train.features['label'].int2str(row)

df_eng_train['label_name'] = df_eng_train['label'].apply(label_int2str)

In [164]:
def label_int2str2(row):
  return eng_dev.features['label'].int2str(row)

df_eng_dev['label_name'] = df_eng_dev['label'].apply(label_int2str2)

In [165]:
def label_int2str3(row):
  return eng_test.features['label'].int2str(row)

df_eng_test['label_name'] = df_eng_test['label'].apply(label_int2str3)

In [166]:
df_eng_test.head()

,label,text,label_name
0,1,Spr....2016 poitan feel happy with my partner ...,Non-anti-LGBT+ content
1,1,R u still with ur partner,Non-anti-LGBT+ content
2,1,excellent movie..no unnecessary drama or scene...,Non-anti-LGBT+ content
3,1,"For those who speak about culture., pre marita...",Non-anti-LGBT+ content
4,1,Best movie and people not understand relations...,Non-anti-LGBT+ content


In [167]:
df_eng_train = df_eng_train.dropna()
df_eng_dev = df_eng_dev.dropna()
df_eng_test = df_eng_test.dropna()

In [168]:
eng_train.reset_format()
eng_dev.reset_format()
eng_test.reset_format()

# Tokenization

In [169]:
from transformers import AutoTokenizer

model_ckpt = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [170]:
def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

In [171]:
eng_train = eng_train.filter(lambda example: example['text'] is not None)
eng_dev = eng_dev.filter(lambda example: example['text'] is not None)
eng_test = eng_test.filter(lambda example: example['text'] is not None)
print( len(eng_train), len(eng_dev), len(eng_test) )

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

3162 791 990


In [172]:
#apply tokenizer across all splits in the corpus
eng_train_encoded = eng_train.map(tokenize, batched=True, batch_size=None)
eng_dev_encoded = eng_dev.map(tokenize, batched=True, batch_size=None)
eng_test_encoded = eng_test.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [173]:
print(eng_train_encoded.column_names)

['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']


# Approach 1: Feature Extraction

In [176]:
from transformers import AutoModel

model_ckpt = "bert-base-multilingual-cased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [179]:
device

device(type='cuda')

In [181]:
def extract_hidden_states(batch):
  #Place model inputs on GPU
  inputs = {k:v.to(device) for k, v in batch.items() 
  if k in tokenizer.model_input_names}

  #Extract last hidden states
  with torch.no_grad():
    last_hidden_state = model(**inputs).last_hidden_state
  
  #Return vector for [CLS] token
  return {"hidden_state": last_hidden_state[:, 0].cpu().numpy()}

In [182]:
eng_train_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [183]:
eng_dev_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [184]:
eng_train_hidden = eng_train_encoded.map(extract_hidden_states, batched=True)
eng_dev_hidden = eng_dev_encoded.map(extract_hidden_states, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [185]:
eng_train_hidden.column_names

['label',
 'text',
 'input_ids',
 'token_type_ids',
 'attention_mask',
 'hidden_state']

In [186]:
X_eng_train = np.array(eng_train_hidden['hidden_state'])
X_eng_dev = np.array(eng_dev_hidden['hidden_state'])

Y_eng_train = np.array(eng_train_hidden['label'])
Y_eng_dev = np.array(eng_dev_hidden['label'])

print( X_eng_train.shape, X_eng_dev.shape, Y_eng_train.shape, Y_eng_dev.shape)

(3162, 768) (791, 768) (3162,) (791,)


## Training an LR Classifier

In [187]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter = 3000)
lr_clf.fit(X_eng_train, Y_eng_train)

LogisticRegression(max_iter=3000)

In [188]:
lr_clf.score(X_eng_dev, Y_eng_dev)

0.922882427307206

In [189]:
#Evaluating a dummy classifier for comparison which always returns the majority class
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_eng_train, Y_eng_train)
dummy_clf.score(X_eng_dev, Y_eng_dev)

0.9241466498103666

In [190]:
Y_eng_preds = lr_clf.predict(X_eng_dev)

In [192]:
tf.math.confusion_matrix(Y_eng_dev, Y_eng_preds)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 10,  48,   0],
       [ 11, 720,   0],
       [  0,   2,   0]], dtype=int32)>

In [194]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_dev, Y_eng_preds, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.48      0.17      0.25        58
Non-anti LGBTQIA+ content       0.94      0.98      0.96       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.92       791
                macro avg       0.47      0.39      0.40       791
             weighted avg       0.90      0.92      0.91       791



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Approach 2: Fine Tuning

In [195]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [196]:
#Define performance metrics

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average = "macro")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [197]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 32
logging_steps = len(eng_train_encoded) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 5,
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [198]:
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = eng_train_encoded,
    eval_dataset = eng_dev_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.252900,0.246458,0.924147,0.320193
2,0.155700,0.217855,0.929204,0.439903
3,0.110300,0.243681,0.932996,0.445004
4,0.068800,0.314468,0.934260,0.441451
5,0.044400,0.321249,0.936789,0.455556


TrainOutput(global_step=495, training_loss=0.12541247154727125, metrics={'train_runtime': 434.2559, 'train_samples_per_second': 36.407, 'train_steps_per_second': 1.14, 'total_flos': 2079911567139840.0, 'train_loss': 0.12541247154727125, 'epoch': 5.0})

In [199]:
preds_output = trainer.predict(eng_dev_encoded)

In [200]:
preds_output.metrics

{'test_accuracy': 0.9367888748419722,
 'test_f1': 0.45555555555555555,
 'test_loss': 0.32124894857406616,
 'test_runtime': 7.6429,
 'test_samples_per_second': 103.494,
 'test_steps_per_second': 3.271}

In [201]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [202]:
tf.math.confusion_matrix(Y_eng_dev, y_preds)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 16,  42,   0],
       [  6, 725,   0],
       [  0,   2,   0]], dtype=int32)>

In [203]:
print(classification_report(Y_eng_dev, y_preds, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.73      0.28      0.40        58
Non-anti LGBTQIA+ content       0.94      0.99      0.97       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.94       791
                macro avg       0.56      0.42      0.46       791
             weighted avg       0.92      0.94      0.92       791



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# EDA

**Saving dataset in a csv format classwise separately**

In [204]:
eng_train.set_format(type="pandas")
eng_dev.set_format(type="pandas")
eng_test.set_format(type="pandas")

In [206]:
df_eng_train = eng_train[:]
df_eng_dev = eng_dev[:]
df_eng_test = eng_test[:]

df_eng_train

,label,text
0,1,"I support her, very smart ponnu"
1,0,priyadharshini kannan same gender attraction ...
2,1,Bro u name and phone number (or)mobile number ...
3,1,experience Thaks bro I love you so much bro
4,1,world is becoming bad day by day....
...,...,...
3157,1,That Akka is really good soul
3158,1,Good madam.if i get any oppertunity in future ...
3159,1,judgment is based on scientific facts
3160,1,Respect to allll🙏🙏🙏


In [207]:
df_homophobic=df_eng_train[df_eng_train.label == 0]
df_homophobic.head()

,label,text
1,0,priyadharshini kannan same gender attraction ...
22,0,Really excellent movie....I feel that no one a...
31,0,Madam read bible
94,0,it's due to abnormalities present sexual dimor...
96,0,i think u too gay....thats y u telling with t...


In [209]:
df_transphobic=df_eng_train[df_eng_train.label == 2]
df_transphobic.head()

,label,text
401,2,Sex the 9.
1090,2,Magalakshmi Mukunthan Ella transgalayum konnud...
1340,2,Today also one transgender scolded me for not ...
1418,2,Rajeswari Rajagopalan ; Sister....last week fo...
2330,2,Hey seriously I thought She was a Transgender


In [211]:
df_homophobic.to_csv('homophobic.txt', sep="\t", header=False, index=False)
df_transphobic.to_csv('transphobic.txt', sep="\t", header=False, index=False)

**Applying EDA to homophobia and transphobia classes**

In [212]:
import nltk

nltk.download('wordnet')
!git clone https://github.com/jasonwei20/eda_nlp.git
%cd eda_nlp/code

!ls

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Cloning into 'eda_nlp'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 396 (delta 6), reused 10 (delta 4), pack-reused 379
Receiving objects: 100% (396/396), 20.42 MiB | 11.22 MiB/s, done.
Resolving deltas: 100% (189/189), done.
/content/eda_nlp/code
augment.py  eda.py  __pycache__


In [213]:
! python augment.py --input=homophobic.txt --num_aug=16

generated augmented sentences with eda for homophobic.txt to eda_homophobic.txt with num_aug=16


In [214]:
! python augment.py --input=transphobic.txt --num_aug=32

generated augmented sentences with eda for transphobic.txt to eda_transphobic.txt with num_aug=32


**Combine augmented data with existing data**

In [221]:
df_homophobic_aug=pd.read_csv('eda_homophobic.txt', sep="\t", names=["label", "text"])
df_transphobic_aug=pd.read_csv('eda_transphobic.txt', sep="\t", names=["label", "text"])

In [222]:
df_homophobic_aug.describe()

,label
count,2669.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [223]:
df_ally=df_eng_train[df_eng_train.label == 1]
df_ally.describe()

,label
count,2999.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [224]:
frames = [df_ally, df_homophobic, df_transphobic, df_homophobic_aug, df_transphobic_aug]
df_aug = pd.concat(frames)

In [225]:
# re shuffling dataframe

df_aug_shuffled = df_aug.sample(frac=1).reset_index(drop=True)
df_aug_shuffled.info

<bound method DataFrame.info of       label                                               text
0         0  it is not mental illness but it is genetical b...
1         1  It's also not opposite to nature... Or modern ...
2         1  Ji... Taboo topic podhum....  Make it once in ...
3         0                              mankind will end soon
4         1                         No comments simply waste 😡
...     ...                                                ...
6024      0  if you woman more get scenes man harmones will...
6025      1       Oh man, I just love her. I love you so much.
6026      1                   Bro back round edited a really a
6027      0  kiran the rebel who will disease people with m...
6028      0  nice movie but india not possible let gay rela...

[6029 rows x 2 columns]>

In [226]:
df_aug_shuffled['label'].value_counts()

1    2999
0    2826
2     204
Name: label, dtype: int64

In [227]:
df_aug_shuffled.to_csv('augmented_data.csv')

# Approach 3: Fine Tuning with EDA

**We convert augmented dataset back to a Datasets object and run tokenization again, before fine-tuning it**

In [230]:
from datasets import Dataset
eng_train_aug = Dataset.from_pandas(df = df_aug_shuffled)

In [231]:
eng_train_aug

Dataset({
    features: ['label', 'text'],
    num_rows: 6029
})

In [232]:
eng_train_aug_encoded = eng_train_aug.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [233]:
eng_train_aug_encoded.column_names

['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']

In [234]:
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = eng_train_aug_encoded,
    eval_dataset = eng_dev_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.198100,0.397780,0.929204,0.451090
2,0.029600,0.451031,0.935525,0.460189
3,0.005700,0.547346,0.931732,0.428032
4,0.002500,0.548947,0.929204,0.441338
5,0.002200,0.558276,0.929204,0.430594


TrainOutput(global_step=945, training_loss=0.03401805857817332, metrics={'train_runtime': 801.6467, 'train_samples_per_second': 37.604, 'train_steps_per_second': 1.179, 'total_flos': 3965776988705280.0, 'train_loss': 0.03401805857817332, 'epoch': 5.0})

In [235]:
preds_output = trainer.predict(eng_dev_encoded)

In [236]:
y_preds_aug = np.argmax(preds_output.predictions, axis=1)

In [237]:
tf.math.confusion_matrix(Y_eng_dev, y_preds_aug)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 13,  45,   0],
       [  8, 722,   1],
       [  0,   2,   0]], dtype=int32)>

In [238]:
print(classification_report(Y_eng_dev, y_preds_aug, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.62      0.22      0.33        58
Non-anti LGBTQIA+ content       0.94      0.99      0.96       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.93       791
                macro avg       0.52      0.40      0.43       791
             weighted avg       0.91      0.93      0.91       791

